In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
import warnings
import numpy as np
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping

warnings.simplefilter(action='ignore', category=FutureWarning)
background = None
accumulated_weight = 0.5

#Creating the dimensions for the ROI...
ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [3]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    _ , thresholded = cv2.threshold(diff, threshold,255,cv2.THRESH_BINARY)
    # Grab the external contours for the image
    contours, hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    else:
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment_max_cont)

In [4]:
cam = cv2.VideoCapture(2)
num_frames = 0
element = 10
num_imgs_taken = 0
kernel = np.ones((2,2),np.uint8)

if not cam.isOpened():
  print ("Could not open cam")
  exit()

temp = 0
num_frames = 0
letter = 'A'

while(1):
  
  ret, frame = cam.read()
  if ret:
    key = cv2.waitKey(1)
    frame = cv2.flip(frame,1)
    ROI = frame[0:300, 170:470]
    
    gray_frame = cv2.cvtColor(ROI, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)

    if num_frames < 60:
      cal_accum_avg(gray_frame, accumulated_weight)
        
    #Time to configure the hand specifically into the ROI...
    elif num_frames <= 300:
      hand = segment_hand(gray_frame)

      if hand is not None:
        thresholded, hand_segment = hand
        cv2.drawContours(frame.copy(), [hand_segment + (0, 0)], -1, (255, 0, 0),1)
        cv2.imshow("Thresholded Hand Image", thresholded)

    else:
      hand = segment_hand(gray_frame)
    
      # Checking if we are able to detect the hand...
      if hand is not None:
        thresholded, hand_segment = hand
        cv2.drawContours(frame.copy(), [hand_segment + (0, 0)], -1, (255, 0, 0),1)
        cv2.imshow("Thresholded Hand Image", thresholded)
        a = 0

        if key==ord('c'):
          print(f'img{temp}')
          mask_name = f"img/{letter}/{temp}_mask.png"
          img_name = f"img/{letter}/{temp}.png"
          cv2.imwrite(mask_name, thresholded)
          cv2.imwrite(img_name,frame)
          temp += 1

    cv2.rectangle(frame.copy(),(0,0),(300,300),(0,255,0),2) 
    num_frames += 1
    cv2.imshow("Sign Detection", frame.copy())
    
    if key==ord('q'):
      break

  if cv2.waitKey(10) & 0xFF == ord('q'):
      break

cam.release()
cv2.destroyAllWindows()